# Google Earth Engine Panel Data Creation

## Initialize

In [25]:
!pip install geemap
#!pip install ee

In [26]:
!pip install uszipcode

In [27]:
#GEE specific
import ee
import geemap
import math

#plotting and functions
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import itertools
from time import time

# Postgres
import psycopg2

# Zip code info
#from uszipcode import SearchEngine

In [28]:
#!earthengine authenticate

In [29]:
#Initialize Google Earth Engine
#ee.Authenticate() #just needed the 1st time
ee.Initialize()

In [30]:
%pip install pyngrok nest_asyncio fastapi uvicorn loguru

Note: you may need to restart the kernel to use updated packages.


In [31]:
from fastapi import FastAPI
from pydantic import BaseModel
from loguru import logger

app = FastAPI()

In [32]:
# eeImage.py

import ee
import math
import itertools

ee.Initialize()

counties = ee.FeatureCollection("TIGER/2018/Counties")
la_county = counties.filter(ee.Filter.eq('NAME', 'Los Angeles'))


def apply_3bands(image, band):
    i_8_bit = image.select(band).toUint8()
    square = ee.Kernel.square(**{'radius': 4})
    entropy = i_8_bit.entropy(square)
    glcm = i_8_bit.glcmTexture(**{'size': 4})
    contrast = glcm.select(str(band)+'_contrast')
    
    # Create a list of weights for a 9x9 kernel.
    list = [1, 1, 1, 1, 1, 1, 1, 1, 1]
    # The center of the kernel is zero.
    centerList = [1, 1, 1, 1, 0, 1, 1, 1, 1]
    # Assemble a list of lists: the 9x9 kernel weights as a 2-D matrix.
    lists = [list, list, list, list, centerList, list, list, list, list]
    # Create the kernel from the weights.
    # Non-zero weights represent the spatial neighborhood.
    kernel = ee.Kernel.fixed(9, 9, lists, -4, -4, False)
    neighs = i_8_bit.neighborhoodToBands(kernel)
    gearys = i_8_bit.subtract(neighs).pow(2).reduce(ee.Reducer.sum()).divide(math.pow(9, 2))
    image = image.addBands(entropy.rename(str(band)+'_Entropy')).addBands(contrast.rename(str(band)+'_Contrast')).addBands(gearys.rename(str(band)+'_Gearys'))   
    return image

def add_neighborhood_bands(image):
    bands = ['R', 'G', 'B', 'N']
    for band in bands:
        image = apply_3bands(image, band)
    return image
    
def add_NDVI(image):
    image = image.addBands(image.normalizedDifference(['N','R']).rename('NDVI'))
    return image


def get_images(param_dict):
    
    
    source_image_collection = param_dict['source_image_collection']
    years = param_dict['years']
#     counties = param_dict['counties']
    counties = {'la_county': la_county}

    image_names = []
    images = []

    combos = list(itertools.product(years, counties.keys()))
    for i in combos:
        year = str(i[0])
        county = i[1]

        image_name = str(i[0])+'_'+i[1]
        image_names.append(image_name)

        image = ee.ImageCollection(source_image_collection)\
                                .filterDate(f'{year}-01-01', f'{year}-12-31')\
                                .select(['R','G','B','N'])\
                                .median().clip(counties[county])
        images.append(image)
        images_with_3band = list(map(add_neighborhood_bands, images))
        images_with_NDVI = list(map(add_NDVI, images_with_3band))
    return dict(zip(image_names, images_with_NDVI))

In [33]:
#areas.py

la_county_income_zipcode2 = ee.FeatureCollection("projects/california-lawn-detection/assets/income_zipcode2019")

la_county_income_zipcode = la_county_income_zipcode2.select(ee.List(['zipcode', '2019zipcod','shape_area']),
                                                            ee.List(['ZipCode', 'Median_Income','Area_sqft']))


def area_calculation(image, class_number, shape, pixel_scale = 20):

    if type(shape) == str:
        shape = la_county_income_zipcode.filter(ee.Filter.eq('ZipCode', shape))

    areaImage = image.eq(class_number).multiply(ee.Image.pixelArea())

    area = areaImage.reduceRegion(
        reducer = ee.Reducer.sum(),
        geometry = shape,
        scale = pixel_scale,
        maxPixels = 1e13)


    area_sq_m = area.getInfo().get('classification')

    area_sq_km = area_sq_m / 1e6

    return area_sq_km



def get_areas(imagery, polygon):

    dictionary = {}

    start = time()
    water_area = area_calculation(imagery, 0, polygon, 20)
    dictionary['water_area'] = water_area

    vegetation_trees_area = area_calculation(imagery, 1, polygon, 20)
    dictionary['tree_area'] = vegetation_trees_area

    vegetation_grass_area = area_calculation(imagery, 2, polygon, 20)
    dictionary['grass_area'] = vegetation_grass_area

    turf_area = area_calculation(imagery, 3, polygon, 20)
    dictionary['turf_area'] = turf_area

    impervious_area = area_calculation(imagery, 4, polygon, 20)
    dictionary['impervious_area'] = impervious_area

    soil_area = area_calculation(imagery, 5, polygon, 20)
    dictionary['soil_area'] = soil_area

    total_area = water_area + vegetation_trees_area + vegetation_grass_area + turf_area + impervious_area + soil_area
    dictionary['polygon_area'] = total_area

    end = time()

    dictionary['inference_time'] = end-start


    return(dictionary)

In [34]:
#eeArea.py

from fastapi import APIRouter
import json

#from eeImage import get_images
#from gee_generate_panel_data_zipcode_db_v0704 import get_images, clf, BANDS, area_calculation, get_areas  
from eeModel import clf, BANDS
#from eeArea import area_calculation, get_areas
import ee

router1 = APIRouter() #ROUTER PREFIX = /polygon_areas
router2 = APIRouter()

params = {
        'source_image_collection' : 'USDA/NAIP/DOQQ',
        'years' : [2020]
         }

images = get_images(params)

rf_model = clf
bands = BANDS
training_image_classified = images['2020_la_county'].select(bands).classify(rf_model)


def get_coordinate_list(json_polygon):
    '''
    returns the area of the coordinate space in the target json file

    in: json data with coordinates under ['path']['Md'] item
    out: list of coordinates

    router extension: /polygon
    e.g. base_url/polygon/{data}
    '''

    coords = json.loads(json_polygon)['path']['Md']
    coord_list = [[float(i['lng']), float(i['lat'])] for i in coords]

    return coord_list


@router1.get("/{json_polygon}")
async def calc_polygon_area(json_polygon):
    coords = get_coordinate_list(json_polygon)

    poly = ee.Geometry.Polygon(coords)

    areas = get_areas(training_image_classified, poly)

    return areas


@router2.get("/{zipcode}")
async def calc_zip_area(zipcode):

    areas = get_areas(training_image_classified, zipcode)

    return areas

In [ ]:

from fastapi import FastAPI, Request
import uvicorn
from pydantic import BaseModel

#custom imports
#from areas import router1
#from areas import router2


app = FastAPI()

#import router to output a classification with a tensorflow model
app.include_router(router1, prefix = '/predict_polygon')
app.include_router(router2, prefix = '/predict_region')

@app.get("/")
async def root():
    return ("WELCOME TO THE LAWN CONVERSION PROJECT")


#if __name__=="__main__":
uvicorn.run("main:app", host='0.0.0.0', port=5000, reload=True)


INFO:     Will watch for changes in these directories: ['/user/mids-w210-capstone/notebooks']
INFO:     Uvicorn running on http://0.0.0.0:5000 (Press CTRL+C to quit)
INFO:     Started reloader process [101] using StatReload
